# Daily data processing and updating

Every day I will download the daily updated file on collected information on cases and deaths due to the COVID-19 virus, courtesy of https://ourworldindata.org/coronavirus-source-data.

After downloading the file manually and placing it in the correct folder I can run this notebook once fully to convert the data to a few dataframes that I will be working with.

Namely for for each country and date that there is information about I will 4 dataframes each containing the following attributes: new cases, new deaths, total cases, total deaths

I will comment on all steps what is happening and regularly print what we're working with for transparency.

### Data import


First we'll need to collect the data, which will be saved in the same directory as this sheet.

In [123]:
import pandas as pd
import data_tools as tools
import datetime

df = pd.read_excel("COVID-19-geographic-disbtribution-worldwide-2020-03-23.xlsx")
df

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
0,2020-03-23,23,3,2020,10,0,Afghanistan,AF
1,2020-03-22,22,3,2020,0,0,Afghanistan,AF
2,2020-03-21,21,3,2020,2,0,Afghanistan,AF
3,2020-03-20,20,3,2020,0,0,Afghanistan,AF
4,2020-03-19,19,3,2020,0,0,Afghanistan,AF
...,...,...,...,...,...,...,...,...
6363,2020-03-20,20,3,2020,0,0,Zambia,ZM
6364,2020-03-19,19,3,2020,2,0,Zambia,ZM
6365,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW
6366,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW


### Data processing

As a first step we will clean out th data a bit since we have quite some values that we do not need.

Considering the 'DateRep' column contains a datetime that Python can translate we will delete the unneeded time attributes as well as the geoid's since we will not be using them.

In [124]:
df = df.drop(columns=['Day', 'Month', 'Year', 'GeoId'])
df

,DateRep,Cases,Deaths,Countries and territories
0,2020-03-23,10,0,Afghanistan
1,2020-03-22,0,0,Afghanistan
2,2020-03-21,2,0,Afghanistan
3,2020-03-20,0,0,Afghanistan
4,2020-03-19,0,0,Afghanistan
...,...,...,...,...
6363,2020-03-20,0,0,Zambia
6364,2020-03-19,2,0,Zambia
6365,2020-03-23,0,0,Zimbabwe
6366,2020-03-22,1,0,Zimbabwe


Next, just to make our next step a little bit easier we will create two lists. One for every date and another for every country that can be found in the database.

In [32]:
dates = []
countries = []

for entry in df['DateRep']:
    if entry not in dates:
        dates.append(entry)

for entry in df['Countries and territories']:
    if entry not in countries:
        countries.append(entry)

Now for the actual data extraction. 
We will start with the new cases and deaths and continue from there.

To create the structures of our dataframes we will make a list filled with tuples, which in turn contain the name of the columns we will create (in this case the 'Date' and all country names) followed by a list that holds the values of cases or deaths of that country. 
We will always follow the order of the dates list we created before when filling the lists of country values to they all line up.

At the end we convert those structure lists into dictionaries to be able to use pandas on them and convert them into dataframes.

In [125]:
structure_cases = [('Date', dates)]
structure_deaths = [('Date', dates)]
i = 1

for country in countries:
    structure_cases.append((country, []))
    structure_deaths.append((country, []))
    
    country_df = df.loc[df['Countries and territories']==country]
    collected_dates = country_df['DateRep'].tolist()
    
    for date in dates:
        if date in collected_dates:
            entry = country_df.loc[country_df['DateRep']==date]
            structure_cases[i][1].append(entry['Cases'].values[0])
            structure_deaths[i][1].append(entry['Deaths'].values[0])
            
        else:
            structure_cases[i][1].append(0)
            structure_deaths[i][1].append(0)
    
    i += 1

dict_cases = {title: content for (title, content) in structure_cases}
dict_deaths = {title: content for (title, content) in structure_deaths}

New we're finlly ready to create the dataframes for both new cases as well as new deaths for the global data.

In [116]:
df_cases_new = pd.DataFrame(dict_cases).set_index('Date')
df_cases_new.name = 'New Cases'

df_cases_new

,Afghanistan,Angola,Albania,Andorra,Argentina,Armenia,Antigua_and_Barbuda,Algeria,Australia,Austria,...,United_Arab_Emirates,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-23,10,0,13,25,41,30,0,8,611,607,...,0,665,6,8459,23,13,0,24,1,0
2020-03-22,0,2,6,13,67,24,0,0,224,375,...,13,1035,3,7123,25,0,0,7,0,1
2020-03-21,2,0,0,0,30,14,0,12,165,453,...,0,706,0,5374,16,10,3,2,0,1
2020-03-20,0,0,11,22,31,7,0,9,144,550,...,27,647,0,4835,15,0,0,9,0,0
2020-03-19,0,0,4,39,18,37,0,13,111,314,...,0,680,2,2988,29,7,0,15,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-03,0,0,0,1,0,0,0,0,4,4,...,0,4,0,14,0,0,0,0,0,0
2020-03-07,0,0,0,0,6,0,0,0,4,33,...,0,48,0,105,0,0,0,1,0,0
2020-03-06,0,0,0,0,1,0,0,5,7,12,...,2,30,0,74,0,0,0,0,0,0


In [117]:
df_deaths_new = pd.DataFrame(dict_deaths).set_index('Date')
df_deaths_new.name = 'New Deaths'

df_deaths_new

,Afghanistan,Angola,Albania,Andorra,Argentina,Armenia,Antigua_and_Barbuda,Algeria,Australia,Austria,...,United_Arab_Emirates,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-23,0,0,0,0,0,0,0,5,0,8,...,0,48,0,131,0,0,0,0,0,0
2020-03-22,0,0,0,0,1,0,0,0,0,2,...,2,56,0,80,0,0,0,0,0,0
2020-03-21,0,0,0,0,0,0,0,3,1,0,...,0,33,0,110,0,0,0,0,0,0
2020-03-20,0,0,0,0,1,0,0,1,0,2,...,0,41,0,0,0,0,0,0,0,0
2020-03-19,0,0,1,0,0,0,0,1,1,1,...,0,43,0,42,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
2020-03-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
2020-03-06,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


Now the last thing that remains is to use the dataframes that we just created to create the dataframes that contain the total amount of cases and deaths.

In [120]:
df_cases_total = df_cases_new 
df_cases_total.name = 'Total Cases'

for country in countries:
    count = 0
    
    for date in reversed(dates):
        count += df_cases_total.at[date, country]
        df_cases_total.at[date, country] = count
        
df_cases_total

,Afghanistan,Angola,Albania,Andorra,Argentina,Armenia,Antigua_and_Barbuda,Algeria,Australia,Austria,...,United_Arab_Emirates,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-23,213,4,2502,576,2919,1707,9,2312,16943,45239,...,3778,66753,29,241714,669,188,265,2899,11,5
2020-03-22,179,2,2413,463,2653,1517,8,2210,15234,41608,...,3625,61070,17,206508,511,142,229,2781,8,3
2020-03-21,155,0,2337,375,2428,1357,7,2116,14136,38584,...,3472,56052,11,179761,376,109,193,2687,6,1
2020-03-20,131,0,2267,300,2270,1221,6,2022,13262,35935,...,3332,52069,8,160137,266,76,157,2600,4,0
2020-03-19,109,0,2197,225,2142,1099,5,1940,12553,33739,...,3192,48792,5,145887,172,53,124,2515,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-03,0,0,0,1,19,0,0,58,120,155,...,30,404,0,704,0,0,0,2,0,0
2020-03-07,0,0,0,0,11,0,0,44,86,95,...,22,277,0,455,0,0,0,1,0,0
2020-03-06,0,0,0,0,3,0,0,30,56,39,...,14,154,0,220,0,0,0,0,0,0


In [119]:
df_deaths_total = df_deaths_new 
df_deaths_total.name = 'Total Deaths'

for country in countries:
    count = 0
    
    for date in reversed(dates):
        count += df_deaths_total.at[date, country]
        df_deaths_total.at[date, country] = count
        
df_deaths_total

,Afghanistan,Angola,Albania,Andorra,Argentina,Armenia,Antigua_and_Barbuda,Algeria,Australia,Austria,...,United_Arab_Emirates,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-23,0,0,2,0,4,0,0,15,7,16,...,2,281,0,471,0,0,0,0,0,0
2020-03-22,0,0,2,0,4,0,0,10,7,8,...,2,233,0,340,0,0,0,0,0,0
2020-03-21,0,0,2,0,3,0,0,10,7,6,...,0,177,0,260,0,0,0,0,0,0
2020-03-20,0,0,2,0,3,0,0,7,6,6,...,0,144,0,150,0,0,0,0,0,0
2020-03-19,0,0,2,0,2,0,0,6,6,4,...,0,103,0,150,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-03,0,0,0,0,0,0,0,0,1,0,...,0,1,0,12,0,0,0,0,0,0
2020-03-07,0,0,0,0,0,0,0,0,1,0,...,0,1,0,8,0,0,0,0,0,0
2020-03-06,0,0,0,0,0,0,0,0,1,0,...,0,1,0,6,0,0,0,0,0,0


### Saving

With all the dataframes created we can now save them to a file to be used to train networks and regression models on.
It will always save on the same file so that all other applications can just rerun when the data is updated.

In [121]:
df_cases_new.to_csv('new_cases.cvs')
df_cases_total.to_csv('total_cases.cvs')

df_deaths_new.to_csv('new_cases.cvs')
df_deaths_total.to_csv('total_cases.cvs')